In [19]:
#import
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import scipy.cluster.hierarchy as hr
from geojson import LineString
import random

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd

import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places

api_key = '5b3ce3597851110001cf62487a4f3ad33ea14791b72150c9e6446e5e'
clnt = client.Client(key=api_key)

In [21]:
#GPS data open and read
a=[]
with open('C:/Users/찬란한 돌멩이/Desktop/GPS DAT/Test DAT/GPS.txt', 'r') as f:
    lines = f.readlines()
#GPS to list
	
    for i in lines:
        b= i.split(sep=',')
        b[6]=b[6].replace("\n","")
        b[6]=b[6].replace(":","")
        a.append(b)

#Pre-processing(위도, 경도, 시간 값만 추출)
n_a = np.array(a)
for i in range(4):
    n_a = np.delete(n_a,2,axis=1)
n_a = n_a.astype(np.float64)
GPS_info_list1 = n_a.tolist()

In [22]:
#DTW 분석을 위한 Dummy 생성
N = len(GPS_info_list1)//100+1
for i in range(N):
    if i*100+100 > len(GPS_info_list1):
        globals()['GPS_dummy_'+str(i)] = GPS_info_list1[i*100:len(GPS_info_list1)+1]
    else:
        globals()['GPS_dummy_'+str(i)] = GPS_info_list1[i*100:i*100+100]


distance_matrix_dtw=np.zeros(shape=(N,N))
path_list=[]

#DTW분석
for i in range(len(GPS_info_list1)//100):
    for j in range(i+1, len(GPS_info_list1)//100+1):
        distance, path = fastdtw(globals()['GPS_dummy_'+str(i)],globals()['GPS_dummy_'+str(j)] , dist=euclidean)
        distance_matrix_dtw[i][j] = distance
        path_list.append(path) 
distance_matrix_dtw = distance_matrix_dtw.tolist()

In [23]:
#Agglomerative clustering
n_clusters=12
agg = AgglomerativeClustering(n_clusters= n_clusters, affinity='precomputed',
                              linkage='average')
u = agg.fit_predict(distance_matrix_dtw)

c:\anaconda3\envs\project23\lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [24]:
#loading Agglomerative Clustering result on differnt lists(클러스터 수만큼의 리스트들)

a=0
b=0
for i in range(n_clusters):
    globals()['Clusters_'+str(i)]=[]
    temp=[]
    
    for j in u:
        if j == a:
            temp.append(b)
        b+=1
    b=0
    globals()['Clusters_'+str(i)] = temp
    a+=1

In [25]:

#1개의 cluster를 임의 선택하여, 그 중 한개의 원소를 임의 추출한 후, 그 값을 정수로 반환.
#반환된 정수 번째의 dummy리스트를 호출하여, Shapely polygon으로 변환
for i in range(n_clusters):
    globals()['path_polygon_'+str(i)]=[]
    for j in  globals()['GPS_dummy_'+str(random.choice(globals()['Clusters_'+str(i)]))]:
         globals()['path_polygon_'+str(i)].append(str(j[1])+" "+str(j[0])+" ")
    globals()['path_polygon_'+str(i)] = ','.join( globals()['path_polygon_'+str(i)])
    globals()['path_polygon_'+str(i)]= 'LineString'+" "+"("+ globals()['path_polygon_'+str(i)] +")"


In [26]:
#polygon을 mapping
wkt_str = globals()['path_polygon_'+str(random.randint(0,n_clusters))]
aoi_geom = wkt.loads(wkt_str)

aoi_coords = list(aoi_geom.coords)
aoi_coords = [(y,x) for x,y in aoi_coords] 
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)
folium.vector_layers.Polygon(aoi_coords,
                                     color='#00d699',
                                     fill_color='#77d699',
                                    fill_opacity=0.2,
                                     weight=3).add_to(m)
m